In [1]:
import importlib

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
model_module = importlib.import_module("handyrl.model")
env_module = importlib.import_module("handyrl.envs.kaggle.hungry_geese")

Loading environment football failed: No module named 'gfootball'


In [3]:
e = env_module.Environment()
e.reset()

In [4]:
obs = e.observation()
obs = torch.from_numpy(obs.reshape(1, 17, 7, 11)).clone()
obs_ = torch.cat([obs, obs], dim=0)
print(f"size: {obs_.size()}, type: {obs_.dtype}")

size: torch.Size([2, 17, 7, 11]), type: torch.float32


In [5]:
input_ = torch.randn(2, 17, 7, 11)
print(f"size: {input_.size()}, type: {input_.dtype}")

size: torch.Size([2, 17, 7, 11]), type: torch.float32


In [6]:
net = env_module.GeeseNet(e)

In [7]:
out = net(obs_)
print(f"size: {out['policy'].size()}, type: {out['policy'].dtype}")

size: torch.Size([2, 4]), type: torch.float32


In [8]:
out

{'policy': tensor([[-0.2113, -0.8816, -1.0376,  1.5567],
         [-0.2113, -0.8816, -1.0376,  1.5567]], grad_fn=<MmBackward>),
 'value': tensor([[0.1120],
         [0.1120]], grad_fn=<TanhBackward>),
 'h_head_p': tensor([[2.7166, 0.0000, 0.8260, 0.0000, 0.0000, 1.8173, 1.2625, 2.5324, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.6857, 0.0000, 0.0000, 1.1608,
          2.7270, 0.0000, 2.2875, 0.0000, 0.5518, 0.9664, 0.0000, 1.6875, 1.2525,
          0.0000, 0.0000, 0.0000, 0.0000, 2.8033],
         [2.7166, 0.0000, 0.8260, 0.0000, 0.0000, 1.8173, 1.2625, 2.5324, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.6857, 0.0000, 0.0000, 1.1608,
          2.7270, 0.0000, 2.2875, 0.0000, 0.5518, 0.9664, 0.0000, 1.6875, 1.2525,
          0.0000, 0.0000, 0.0000, 0.0000, 2.8033]], grad_fn=<SumBackward1>),
 'h_head_v': tensor([[0.0000, 0.0000, 0.0000, 0.9924, 0.0000, 0.0000, 1.0137, 0.8023, 0.0000,
          0.3823, 0.0000, 0.8006, 1.8188, 0.0000, 1.4493, 1.4238, 0.3792,

In [9]:
# pytorch_total_params
params = sum(p.numel() for p in net.parameters())
print(f"{params:,}")

146,880


In [10]:
# pytorch_total_params (trainable)
params = sum(p.numel() for p in net.parameters() if p.requires_grad)
print(f"{params:,}")

146,880


In [11]:
net

GeeseNet(
  (conv0): TorusConv2d(
    (conv): Conv2d(17, 32, kernel_size=(3, 3), stride=(1, 1))
    (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (blocks): ModuleList(
    (0): TorusConv2d(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
      (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): TorusConv2d(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
      (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (2): TorusConv2d(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
      (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (3): TorusConv2d(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
      (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (4): TorusConv2d(
      (conv): Conv2d(32, 32,